In [ ]:
import numpy as np
from sklearn.cluster import KMeans, OPTICS
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from matplotlib import animation
import ipywidgets as widgets
%matplotlib widget

In [ ]:
starlink_data = np.load("starlink_data.npy")
starlink_data = np.nan_to_num(starlink_data, nan = 0)
print(starlink_data.shape)

In [ ]:
def kmeans_elbow(data: np.ndarray, num_cluster: int):
    '''
    Calculates the sum of squares for different number of clusters using K-means
    This allows to select an optimal number of cluster
    '''
    sum_squares = np.zeros(len(range(2, num_cluster+1, 2)))
    j=0
    for i in range(2, num_cluster+1, 2):
        kmeans = KMeans(n_clusters=i, random_state=0).fit(data)
        print("Num cluster:", i, "Sum of squares:", kmeans.inertia_)
        sum_squares[j] = kmeans.inertia_
        j+=1
    
    plt.figure(figsize=(12, 7.5))
    plt.plot(np.arange(2, num_cluster+1, 2), sum_squares)
    plt.ylabel('Sum of Squares', fontsize = 14)
    plt.xlabel('Number of clusters', fontsize = 14)
    plt.show()

In [ ]:

sat_ind = np.random.choice(starlink_data.shape[0],500)
time = starlink_data[sat_ind, :, 0]

time = (time - 2022*10**9) *10**-6
time = time - time[:,0:1]
time[time<0] = 0


angular_momentum_x = starlink_data[sat_ind, :, 7]
angular_momentum_y = starlink_data[sat_ind, :, 8]
angular_momentum_z = starlink_data[sat_ind, :, 9]

normed_angular_momentum_x = StandardScaler().fit_transform(angular_momentum_x)
normed_angular_momentum_y = StandardScaler().fit_transform(angular_momentum_y)
normed_angular_momentum_z = StandardScaler().fit_transform(angular_momentum_z)

print(time.shape)
print(normed_angular_momentum_x.shape)



In [ ]:
kmeans_elbow(np.transpose(normed_angular_momentum_x), num_cluster=20)
kmeans_elbow(np.transpose(normed_angular_momentum_y), num_cluster=20)
kmeans_elbow(np.transpose(normed_angular_momentum_z), num_cluster=20)


In [ ]:
optics = OPTICS()
optics.fit(normed_angular_momentum_x)
print(len(np.unique(optics.labels_)))
optics.fit(normed_angular_momentum_y)
print(len(np.unique(optics.labels_)))
optics.fit(normed_angular_momentum_z)
print(len(np.unique(optics.labels_)))


In [ ]:
radius = np.linalg.norm(starlink_data[:, :, 1:4], axis=2)
print(radius.shape)

kmeans_elbow(radius, num_cluster=20)

In [ ]:
def plot_time_clusters(cluster: np.ndarray, num_clusters: int, 
            angular_momentum_x: np.ndarray,
            angular_momentum_y: np.ndarray,
            angular_momentum_z: np.ndarray,
            time: np.ndarray, axis: int):
    fig = plt.figure(figsize= (15,15))
    fig, axs =plt.subplots(3,1, sharex= True)
    for i in range(0,num_clusters):
        title = "cluster" + str(i)
        if axis == 0:
            time_tmp = time[cluster == i]
            angular_momentum_x_tmp = angular_momentum_x[cluster == i]
            angular_momentum_y_tmp = angular_momentum_y[cluster == i]
            angular_momentum_z_tmp = angular_momentum_z[cluster == i]
            ind  = np.dstack(np.random.choice(time_tmp.shape[0],1))#,np.arange(0,time.shape[1])])
        elif axis == 1:
            time_tmp = time[:,cluster == i]
            angular_momentum_x_tmp = angular_momentum_x[:,cluster == i]
            angular_momentum_y_tmp = angular_momentum_y[:,cluster == i]
            angular_momentum_z_tmp = angular_momentum_z[:,cluster == i]
            # ind  = np.random.choice(time.shape[0],5)
            ind = np.arange(0,time_tmp.shape[0],100)
        axs[0].scatter(time_tmp[ind,:],angular_momentum_x_tmp[ind,:])
        axs[0].set_ylabel('h_x')
        axs[1].scatter(time_tmp[ind,:],angular_momentum_y_tmp[ind,:])
        axs[1].set_ylabel('h_y')
        axs[2].scatter(time_tmp[ind,:],angular_momentum_z_tmp[ind,:],label = title)
        axs[2].set_ylabel('h_z')
        axs[2].legend()

In [ ]:
def plot_clusters_3d( time_point: int, 
    clusters: np.ndarray, num_clusters: int,           
            angular_momentum_x: np.ndarray,
            angular_momentum_y: np.ndarray,
            angular_momentum_z: np.ndarray,  axis: int, ax):
    plt.cla()
    if axis == 0:
        for i in range(0,num_clusters):
            title = "Cluster" + str(i)
            # ind  = np.arange(0,angular_momentum_x[clusters == i].shape[1],100)
            ind = time_point
            ax.scatter(angular_momentum_x[clusters == i][:,ind],angular_momentum_y[clusters == i][:,ind],angular_momentum_z[clusters == i][:,ind], s = 40 , label = title)
    elif axis == 1:
        for i in range(0,num_clusters):
            title = "Cluster" + str(i)
            ind  = np.arange(0,angular_momentum_x[:,clusters == i].shape[0],100)
            ax.scatter(angular_momentum_x[:,clusters == i][ind],angular_momentum_y[:,clusters == i][ind],angular_momentum_z[:,clusters == i][ind], s = 40 , label = title)
    ax.set_xlabel('h_x')
    ax.set_ylabel('h_y')
    ax.set_zlabel('h_z')
    ax.set_xlim([-50000, 50000])
    ax.set_ylim([-50000, 50000])
    ax.set_zlim([30500, 32000])
    ax.legend()

In [ ]:
kmeans = KMeans(n_clusters=8, random_state=0)
x_clusters = kmeans.fit_predict(angular_momentum_x)
y_clusters = kmeans.fit_predict(angular_momentum_y)
z_clusters = kmeans.fit_predict(angular_momentum_z)

fig1 = plt.figure(figsize = (10,10))
ax1 = fig1.add_subplot(111, projection='3d')
plot_clusters_3d(0,x_clusters,8,angular_momentum_x,angular_momentum_y,angular_momentum_z,0, ax1)
plt.ion()
plt.show()
plt.ioff()


In [ ]:
plt.close()


plot_time_clusters(x_clusters, 8, angular_momentum_x,angular_momentum_y,angular_momentum_z, time, 0)

In [ ]:
plt.close()
print(time[:,::20].shape[1])
ts_fig = plt.figure(figsize = (10, 10))
ax = ts_fig.add_subplot(111, projection='3d')

anim = animation.FuncAnimation(ts_fig, 
                                plot_clusters_3d, 
                                frames=time.shape[1], 
                                interval=20, 
                                fargs=(z_clusters,8,angular_momentum_x,angular_momentum_y,angular_momentum_z,0,ax),
                                blit=True,
                                repeat_delay=200)

plt.show()

In [ ]:
fig2 = plt.figure(figsize = (10,10))
ax2 = fig2.add_subplot(111, projection='3d')
plot_clusters_3d(0,y_clusters,8,angular_momentum_x,angular_momentum_y,angular_momentum_z,0,ax2)
plt.ion()
plt.show()

plot_time_clusters(y_clusters, 8, angular_momentum_x,angular_momentum_y,angular_momentum_z, time, 0)

In [ ]:
fig3 = plt.figure(figsize = (10,10))
ax3 = fig3.add_subplot(111, projection='3d')
plot_clusters_3d(0,z_clusters,8,angular_momentum_x,angular_momentum_y,angular_momentum_z,0,ax3)
plt.ion()
plt.show()

plot_time_clusters(z_clusters, 8, angular_momentum_x,angular_momentum_y,angular_momentum_z, time, 0)

In [ ]:
x_clusters = kmeans.fit_predict(np.transpose(angular_momentum_x))
y_clusters = kmeans.fit_predict(np.transpose(angular_momentum_y))
z_clusters = kmeans.fit_predict(np.transpose(angular_momentum_z))


plot_clusters_3d(0,x_clusters,8,angular_momentum_x,angular_momentum_y,angular_momentum_z,1)
plot_time_clusters(x_clusters, 8, angular_momentum_x,angular_momentum_y,angular_momentum_z, time, 1)

plot_clusters_3d(0,y_clusters,8,angular_momentum_x,angular_momentum_y,angular_momentum_z,1)
plot_time_clusters(y_clusters, 8, angular_momentum_x,angular_momentum_y,angular_momentum_z, time, 1)

plot_clusters_3d(0,z_clusters,8,angular_momentum_x,angular_momentum_y,angular_momentum_z,1)
plot_time_clusters(z_clusters, 8, angular_momentum_x,angular_momentum_y,angular_momentum_z, time, 1)

In [ ]:
# PLOT RADIUS AS A FUNCTION OF TIME
# PLOT AVERAGE OF  EACH CLUSTER


In [ ]:
fig = plt.figure(figsize = (15,15))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(angular_momentum_x[0,:],angular_momentum_y[0,:],angular_momentum_z[0,:], s = 40 )
ax.set_xlabel('h_x')
ax.set_ylabel('h_y')
ax.set_zlabel('h_z')
plt.show()